In [ ]:
import re

with open('F.txt', 'r', encoding='utf-8') as f:
    content = f.read()

import json


# 正则表达式模式来匹配 LaTeX token，包括换行符
token_pattern = re.compile(r'''
    (\\begin\{[^{}]*\}|\\end\{[^{}]*\}    # 匹配 \begin{} 和 \end{} 结构
    | \\[a-zA-Z]+                         # 匹配 LaTeX 命令，如 \sum, \leq 等
    | \\\\                                # 匹配双反斜杠 \\
    | \\[{}#%&_$|]                        # 匹配单个特殊字符，如 \{, \}, \#, \%, \&, \_, \$, \|
    | \\[!,:;]                            # 匹配单个标点符号，如 \!, \,, \:, \;
    | \'[\^]?                             # 匹配单引号和可能的上标符号，如 \', \'^\ 等
    | \^                                  # 匹配上标符号 ^
    | \_                                  # 匹配下标符号 _
    | \n                                  # 匹配换行符
    | \s+                                 # 匹配一个或多个空格
    | [-+*=&/%<>!?.,;:'"()]               # 匹配原有的符号
    | [0-9]                               # 匹配单个数字
    | \$\$                                # 匹配 $$
    | \S)                                 # 匹配任何非空白字符
''', re.VERBOSE)


token_to_id = {  
    "PAD": 0,  
    "BOS": 1,  
    "EOS": 2
}  
  
tokens = set(token_pattern.findall(content))
  
for idx, token in enumerate(sorted(tokens), start=3):  
    token_to_id[token] = idx  
  
id_to_token = {idx: token for token, idx in token_to_id.items()}  
  
with open('token_to_id.json', 'w', encoding='utf-8') as json_file:  
    json.dump(token_to_id, json_file, ensure_ascii=False, indent=4)

In [ ]:
import json

class Words:
    def __init__(self, token_to_id_path):
        with open(token_to_id_path, 'r', encoding='utf-8') as json_file:
            self.token_to_id = json.load(json_file)
        print(f"共有{len(self.token_to_id)}个token")

    def __len__(self):
        return len(self.token_to_id)

    def encode(self, tokens_list):
        encoded_inputs = {'input_ids': [], 'attention_mask': []}
        for tokens in tokens_list:
            number_list = []
            i = 0
            while i < len(tokens):
                match = None
                # 尝试匹配最长的子字符串
                for j in range(len(tokens), i, -1):
                    substr = tokens[i:j]
                    if substr in self.token_to_id:
                        match = substr
                        break
                # 如果找到匹配，添加对应的数字
                if match:
                    number_list.append(self.token_to_id[match])
                    i += len(match)
                else:
                    # 如果没有匹配，使用单个字符或UNK（默认UNK为3）
                    number_list.append(self.token_to_id.get(tokens[i], self.token_to_id.get("UNK", 3)))
                    i += 1


            encoded_inputs['input_ids'].append(number_list)
            encoded_inputs['attention_mask'].append([1] * len(number_list))
        
        return encoded_inputs